In [1]:
### THRID APPROACH IN WHICH WE HAVE USED ENSEMBLE OF RANDOM FOREST REGRESSION AND XGBOOST FOR ACCURACY BOOSTING AND LIGHTGBM REGRESSOR WITH REGULARIZATION USING RIDGE REGRESSION


In [ ]:
!pip install xgboost lightgbm shap scikit-learn pandas numpy


In [ ]:
import pandas as pd
import numpy as np
import shap

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from sklearn.ensemble import RandomForestRegressor, StackingRegressor
from sklearn.linear_model import Ridge, Lasso
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor


In [ ]:
df = pd.read_csv("ml_dataset.csv")

X = df.drop(columns=["target_spend","customer_id","monetary"])
y = df["target_spend"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [ ]:
def evaluate(y_true, y_pred, name):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    print(f"{name} → RMSE:{rmse:.2f}  MAE:{mae:.2f}  R2:{r2:.2f}")


WAY1 RANDOM+RIDGE+LASSO


In [ ]:
rf_params = {
    "n_estimators":[100,200],
    "max_depth":[5,10],
    "min_samples_split":[2,5]
}

rf_search = RandomizedSearchCV(RandomForestRegressor(), rf_params, n_iter=5, cv=3)
rf_search.fit(X_train, y_train)

rf_best = rf_search.best_estimator_
rf_pred = rf_best.predict(X_test)
evaluate(y_test, rf_pred, "RF Tuned")


RF Tuned → RMSE:232272.73  MAE:16362.67  R2:0.19


In [ ]:
xgb_params = {
    "n_estimators":[100,200],
    "max_depth":[3,5],
    "learning_rate":[0.01,0.1]
}

xgb_search = RandomizedSearchCV(XGBRegressor(), xgb_params, n_iter=5, cv=3)
xgb_search.fit(X_train, y_train)

xgb_best = xgb_search.best_estimator_


In [ ]:
stack_rf_xgb = StackingRegressor(
    estimators=[("rf", rf_best), ("xgb", xgb_best)],
    final_estimator=Ridge()
)

stack_rf_xgb.fit(X_train, y_train)
stack_pred = stack_rf_xgb.predict(X_test)
evaluate(y_test, stack_pred, "RF + XGB Ensemble")


RF + XGB Ensemble → RMSE:191708.14  MAE:13704.47  R2:0.45


In [ ]:
lgb_params = {
    "n_estimators":[100,200],
    "learning_rate":[0.01,0.1],
    "num_leaves":[31,50]
}

lgb_search = RandomizedSearchCV(LGBMRegressor(), lgb_params, n_iter=5, cv=3)
lgb_search.fit(X_train, y_train)

lgb_best = lgb_search.best_estimator_


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000266 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1018
[LightGBM] [Info] Number of data points in the train set: 3136, number of used features: 4
[LightGBM] [Info] Start training from score 29115.352257
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000126 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1019
[LightGBM] [Info] Number of data points in the train set: 3136, number of used features: 4
[LightGBM] [Info] Start training from score 26691.333378
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000793 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1019
[LightGBM] [Info] Number of data points in the train set: 3136, number of used features: 4
[LightGBM] [Info] Start t

In [ ]:
full_stack = StackingRegressor(
    estimators=[
        ("rf", rf_best),
        ("xgb", xgb_best),
        ("lgb", lgb_best)
    ],
    final_estimator=Ridge()
)

full_stack.fit(X_train, y_train)
full_stack_pred = full_stack.predict(X_test)
evaluate(y_test, full_stack_pred, "RF + XGB + LGBM Ensemble")


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000148 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1019
[LightGBM] [Info] Number of data points in the train set: 4704, number of used features: 4
[LightGBM] [Info] Start training from score 28229.687642
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000125 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1018
[LightGBM] [Info] Number of data points in the train set: 3763, number of used features: 4
[LightGBM] [Info] Start training from score 27950.679300
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000129 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1019
[LightGBM] [Info] Number of data points in the train set: 3763, number of used features: 4
[LightGBM] [Info] Start t

In [ ]:
full_stack_lasso = StackingRegressor(
    estimators=[
        ("rf", rf_best),
        ("xgb", xgb_best),
        ("lgb", lgb_best)
    ],
    final_estimator=Lasso()
)

full_stack_lasso.fit(X_train, y_train)
lasso_stack_pred = full_stack_lasso.predict(X_test)
evaluate(y_test, lasso_stack_pred, "Full Ensemble Lasso")


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000179 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1019
[LightGBM] [Info] Number of data points in the train set: 4704, number of used features: 4
[LightGBM] [Info] Start training from score 28229.687642
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000121 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1018
[LightGBM] [Info] Number of data points in the train set: 3763, number of used features: 4
[LightGBM] [Info] Start training from score 27950.679300
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000123 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1019
[LightGBM] [Info] Number of data points in the train set: 3763, number of used features: 4
[LightGBM] [Info] Start t

In [ ]:
ridge_params = {
    "alpha": [0.01, 0.1, 1.0, 10.0, 50.0]
}

ridge_search = RandomizedSearchCV(
    Ridge(),
    ridge_params,
    n_iter=5,
    cv=3
)

ridge_search.fit(X_train, y_train)
ridge_best = ridge_search.best_estimator_


In [ ]:
full_stack = StackingRegressor(
    estimators=[
        ("rf", rf_best),
        ("xgb", xgb_best),
        ("lgb", lgb_best)
    ],
    final_estimator=ridge_best,
    n_jobs=-1
)


In [ ]:
full_stack.fit(X_train, y_train)
full_stack_pred = full_stack.predict(X_test)


In [ ]:
evaluate(y_test, full_stack_pred, "FINAL STACK (RIDGE TUNED)")


FINAL STACK (RIDGE TUNED) → RMSE:197192.96  MAE:14623.04  R2:0.41
